# Simplificando um acórdão

In [1]:
from langchain_ollama import ChatOllama
from llms import get_llama
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyMuPDFLoader
from langchain_core.prompts import PromptTemplate
from langchain_chroma import Chroma

from config import Config

In [2]:
def split_documents(file_path, page_start=None, page_end=None):
    loader = PyMuPDFLoader(file_path=file_path)
    doc = loader.load()[page_start:page_end]

    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=Config.SPLITTER_CHUNK_SIZE,
        chunk_overlap=Config.SPLITTER_CHUNK_OVERLAP
    )
    docs = text_splitter.split_documents(doc)
    return docs

In [3]:
from pprint import pprint
from langchain_ollama import OllamaEmbeddings
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain


def stuff(docs, host, results, key, prompt):
    llm: ChatOllama = get_llama(host=host, model=1, log_callbacks=True)

    embeddings_model = OllamaEmbeddings(
        base_url=Config.OLLAMA_EMBEDDINGS_BASE_URL,
        model=Config.OLLAMA_EMBEDDINGS_MODEL
    )
    vectorstore = Chroma(
        collection_name='acordaos',
        embedding_function=embeddings_model,
    )
    vectorstore.add_documents(documents=docs)
    retriever = vectorstore.as_retriever()

    document_chain = create_stuff_documents_chain(llm, prompt)
    retrieval_chain = create_retrieval_chain(retriever, document_chain)

    # print(f"calling {host}")

    chunks = []

    for chunk in retrieval_chain.stream({
        'input': 'Comece.'
    }):
        answer = chunk.get('answer', '')
        chunks.append(answer)
        print(answer, end='', flush=True)
    print()
    result = "".join(chunks)
    # result = format_chain.invoke({'context': result['output_text']})

    # print('\a')
    #print(f"\nHOST {host}:\n")
    #pprint(result)

    # simplificar_prompt = PromptTemplate.from_template(
    #     '''
    #     Simplifique a linguagem usada no texto. Para isso, siga as seguintes diretrizes
    #     '''
    # )

    results[key]['response'] = result
    results[key]['prompt'] = [
        str(prompt),
    ]

    # history.save('mapreduce', '', result['text'])
    return result

In [4]:
results = {
    "cabecalho": {},
    "relatorio": {},
    "voto": {},
    "decisao": {},
}

In [5]:
simplificados = {}

In [6]:
file_path = "documentos/acordaos/0600012-49_REl_28052024_1.pdf"

## Extraindo o cabeçalho

In [7]:
start_page = 0
end_page = 1
doc = split_documents(file_path, start_page, end_page)

### Prompt do cabeçalho

In [8]:
cabecalho_prompt = PromptTemplate.from_template(
    """
    Extraia do texto encontrado no contexto somente as seguintes informações:

    -   Número do processo (ou número do recurso eleitoral)
    -   Origem
    -   Relator
    -   Recorrente
    -   Advogados(as)
    -   Recorrido
    -   Assunto

    Formate sua resposta como uma lista, assim:

    -   Número do processo: Número do processo
    -   Origem: Origem
    -   Relator: Relator
    -   Recorrente: Recorrente
    -   Advogados: Advogados(as)
    -   Recorrido: Recorrido
    -   Assunto: Assunto, em 2 ou 3 palavras

    ### Exemplo de entrada:

    ```
    PODER JUDICIÁRIO
    TRIBUNAL REGIONAL ELEITORAL DO CEARÁ

    RECURSO ELEITORAL N. 0600012-49.2024.6.06.0033
    ORIGEM: CANINDÉ/CE
    RELATOR: DESEMBARGADOR ELEITORAL DANIEL CARVALHO CARNEIRO
    RECORRENTE: ANTÔNIO ILOMAR VASCONCELOS CRUZ
    ADVOGADOS(AS): FRANCISCO JARDEL RODRIGUES DE SOUSA - OAB CE32787-A,
    LIDENIRA CAVALCANTE MENDONÇA VIEIRA - OAB CE0016731
    RECORRIDO: MINISTÉRIO PÚBLICO ELEITORAL
    ```

    ### Exemplo de saída:

    ```
    -   Número do processo: 0600012-49.2024.6.06.0033
    -   ORIGEM: CANINDÉ/CE
    -   RELATOR: DESEMBARGADOR ELEITORAL DANIEL CARVALHO CARNEIRO
    -   RECORRENTE: ANTÔNIO ILOMAR VASCONCELOS CRUZ
    -   ADVOGADOS(AS): FRANCISCO JARDEL RODRIGUES DE SOUSA - OAB CE32787-A, LIDENIRA CAVALCANTE MENDONÇA VIEIRA - OAB CE0016731
    -   RECORRIDO: MINISTÉRIO PÚBLICO ELEITORAL
    ```

    ### Contexto

    ```
    {context}
    ```

    ### Fim do contexto

    ### Resposta
    """
)

### Recebendo a resposta

In [9]:
result = stuff(doc, 0, results, 'cabecalho', cabecalho_prompt)

Number of requested results 4 is greater than number of elements in index 1, updating n_results = 1



Prompt: Human: 
    Extraia do texto encontrado no contexto somente as seguintes informações:

    -   Número do processo (ou número do recurso eleitoral)
    -   Origem
    -   Relator
    -   Recorrente
    -   Advogados(as)
    -   Recorrido
    -   Assunto

    Formate sua resposta como uma lista, assim:

    -   Número do processo: Número do processo
    -   Origem: Origem
    -   Relator: Relator
    -   Recorrente: Recorrente
    -   Advogados: Advogados(as)
    -   Recorrido: Recorrido
    -   Assunto: Assunto, em 2 ou 3 palavras

    ### Exemplo de entrada:

    ```
    PODER JUDICIÁRIO
    TRIBUNAL REGIONAL ELEITORAL DO CEARÁ

    RECURSO ELEITORAL N. 0600012-49.2024.6.06.0033
    ORIGEM: CANINDÉ/CE
    RELATOR: DESEMBARGADOR ELEITORAL DANIEL CARVALHO CARNEIRO
    RECORRENTE: ANTÔNIO ILOMAR VASCONCELOS CRUZ
    ADVOGADOS(AS): FRANCISCO JARDEL RODRIGUES DE SOUSA - OAB CE32787-A,
    LIDENIRA CAVALCANTE MENDONÇA VIEIRA - OAB CE0016731
    RECORRIDO: MINISTÉRIO PÚBLICO ELEITORA

## Resumindo o relatório

In [10]:
start_page = 2
end_page = 5
doc = split_documents(file_path, start_page, end_page)

### Prompt do relatório

In [11]:
relatorio_prompt = PromptTemplate.from_template(
    """
    Você é um especialista jurídico com foco em simplificação de textos legais. Sua tarefa é analisar acórdãos judiciais e gerar um resumo simplificado, acessível ao público geral, sem perder a precisão jurídica. O formato do documento simplificado deve ser claro, objetivo e seguir uma estrutura pré-definida.
    Preencha o acórdão simplificado utilizando as informações fornecidas em cada bloco. Siga o formato abaixo para garantir que todos os elementos necessários sejam cobertos e organizados conforme a estrutura do acórdão:

    ### FORMATO
    ```
    **Relatório (O Caso)**: Resuma de forma objetiva os fatos apresentados no acórdão, destacando em 3 parágrafos, com no máximo 3 linhas cada um, e, quando for o caso, caixa de texto explicativa, tudo em texto contínuo.

    Informações iniciais do processo analisado pelo juiz: indique o que o autor do recurso pediu e o que o réu alegou para se defender.
    Decisão do juiz no processo inicial: Apresente a decisão do juiz no processo inicial, descrevendo o que o juiz decidiu e as justificativas legais usadas.
    Artigos de lei e fundamentos jurídicos relevantes: cite artigos de lei e fundamentos jurídicos relevantes..
    Quem recorreu e o que alegou: indique quem recorreu à decisão e o que alegou para recorrer.

    Caixa de texto explicativa com termos jurídicos relevantes para a compreensão do assunto principal: forneça definições e explicações simples de termos, expressões ou assuntos jurídicos relevantes para a compreensão do assunto principal. Exemplo: “Propaganda antecipada negativa: A propaganda eleitoral antecipada negativa acontece quando, antes de 16 de agosto do ano eleitoral (art. 36 da Lei nº 9.504/1997), alguém faz críticas para prejudicar adversários políticos e influenciar eleitores. Essa prática é proibida e pode resultar em multa”.
    ```
    ### FIM DO FORMATO

    ### CONTEXTO
    ```
    {context}
    ```
    ### FIM DO CONTEXTO

    Resumo simplificado:
    """
)

### Recebendo a resposta

In [ ]:
result = stuff(doc, 0, results, 'relatorio', relatorio_prompt)


Prompt: Human: 
    Você é um especialista jurídico com foco em simplificação de textos legais. Sua tarefa é analisar acórdãos judiciais e gerar um resumo simplificado, acessível ao público geral, sem perder a precisão jurídica. O formato do documento simplificado deve ser claro, objetivo e seguir uma estrutura pré-definida.
    Preencha o acórdão simplificado utilizando as informações fornecidas em cada bloco. Siga o formato abaixo para garantir que todos os elementos necessários sejam cobertos e organizados conforme a estrutura do acórdão:

    ### FORMATO
    ```
    **Relatório (O Caso)**: Resuma de forma objetiva os fatos apresentados no acórdão, destacando em 3 parágrafos, com no máximo 3 linhas cada um, e, quando for o caso, caixa de texto explicativa, tudo em texto contínuo.

    Informações iniciais do processo analisado pelo juiz: indique o que o autor do recurso pediu e o que o réu alegou para se defender.
    Decisão do juiz no processo inicial: Apresente a decisão do juiz

In [14]:
get_llama(0).get_num_tokens(results['relatorio']['response'])

/home/rafael-albuquerque/LIODS/repos/simplificacao/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
None of PyTorch, TensorFlow >= 2.0, or Flax have been found. Models won't be available and only tokenizers, configuration and file/data utilities can be used.
Token indices sequence length is longer than the specified maximum sequence length for this model (1176 > 1024). Running this sequence through the model will result in indexing errors


1176

In [51]:
simplificar_relatorio_prompt = PromptTemplate.from_template(
    """
    Você é um especialista jurídico com foco em simplificação de documentos legais. Sua missão é reescrever documentos legais em linguagem simplificada, visando o entendimento de pessoas não familiarizadas com o linguajar jurídico. Mantenha a estrutura dos documentos que receber, preservando os títulos e subtítulos, exceto a conclusão, entre outros componentes estruturais do texto. Preserve as informações principais do texto original de forma fiel.

    ### CONTEXTO
    ```
    {context}
    ```
    ### FIM DO CONTEXTO

    Texto simplificado:
    """
)

In [52]:
simplificar_chain = simplificar_relatorio_prompt | get_llama(0)
result = simplificar_chain.invoke({'context': results['relatorio']['response']})

In [53]:
simplificados['relatorio'] = result.content
print(simplificados['relatorio'])

**Relatório (O Caso)**

Um caso envolveu uma ação eleitoral contra o candidato Antônio Ilomar Vasconcelos Cruz, que foi acusado de realizar propaganda eleitoral antecipada antes do período oficial de campanha. O Ministério Público Eleitoral apresentou um recurso pedindo que o Tribunal Regional Eleitoral do Ceará negasse provimento ao recurso.

**Informações iniciais do processo**

O autor do recurso pediu que o tribunal negasse provimento ao recurso, argumentando que não havia sido cometido ato típico de propaganda eleitoral antecipada. O réu alegou que não havia realizado nenhuma conduta ilícita e que sua pré-candidatura foi lançada de forma regular.

**Decisão do juiz no processo inicial**

O juiz decidiu que o candidato Antônio Ilomar Vasconcelos Cruz havia cometido ato típico de propaganda eleitoral antecipada, pois realizou uma apresentação com um jingle de campanha que expressava pedido explícito de votos. Além disso, a infraestrutura do evento e o local onde foi realizado foram 

## Resumindo o voto

In [24]:
start_page = 5
end_page = 11
doc = split_documents(file_path, start_page, end_page)

### Prompt do voto

In [25]:
voto_prompt = PromptTemplate.from_template(
    """
    Você é um especialista jurídico com foco em simplificação de textos legais. Sua tarefa é analisar acórdãos judiciais e gerar um resumo simplificado, acessível ao público geral, sem perder a precisão jurídica. O formato do documento simplificado deve ser claro, objetivo e seguir uma estrutura pré-definida.
    Preencha o acórdão simplificado utilizando as informações fornecidas em cada bloco. Siga o formato abaixo para garantir que todos os elementos necessários sejam cobertos e organizados conforme a estrutura do acórdão:

    ### FORMATO
    ```
    **Voto (Argumentação/ Motivação/ Fundamentação)**: Apresente as razões jurídicas que embasaram a decisão do Tribunal, destacando, em 2 parágrafos, com no máximo 3 linhas, de texto contínuo.

    Princípios legais aplicados: quais foram os princípios legais aplicados.
    Interpretação do Tribunal: A interpretação dada à legislação.
    Argumentos relevantes: Argumentos relevantes usados na motivação.**

    (Use uma linguagem simples e evite jargões técnicos sempre que possível)

    Fundamentação de Fato e de Direito: Explique os fatos relevantes para a decisão e apresente a fundamentação jurídica, citando artigos de lei, precedentes jurisprudenciais que embasam a decisão do Tribunal.
    Conclusão e Decisão: Conclusão do Voto.
    ```
    ### FIM DO FORMATO

    ### CONTEXTO
    ```
    {context}
    ```
    ### FIM DO CONTEXTO

    Resumo simplificado:
    """
)

### Recebendo a resposta

In [26]:
result = stuff(doc, 0, results, 'voto', voto_prompt)

**Voto (Argumentação/ Motivação/ Fundamentação)**

O Tribunal Regional Eleitoral do Ceará decidiu negar provimento ao recurso do candidato Antônio Ilomar Vasconcelos Cruz, que alegou que sua pré-candidatura ao cargo de prefeito de Canindé/CE foi afetada pela propaganda eleitoral antecipada. A decisão foi tomada com base na interpretação da lei e nos princípios de igualdade entre os candidatos.

O Tribunal considerou que o evento realizado em benefício do recorrente, no contexto do lançamento de sua pré-candidatura, apresentava características de propaganda eleitoral antecipada, incluindo a presença de materiais típicos de campanha e uma estrutura significativa. Além disso, as evidências trazidas aos autos mostravam que o evento foi realizado com um objetivo claro de influenciar a opinião dos eleitores e promover a imagem política do recorrente.

**Princípios legais aplicados:** Igualdade entre os candidatos, propaganda eleitoral antecipada

**Interpretação do Tribunal:** O Tribunal con

## Resumindo a decisão

In [34]:
start_page = 10
end_page = 12
doc = split_documents(file_path, start_page, end_page)

### Prompt da decisão

In [35]:
decisao_prompt = PromptTemplate.from_template(
    """
    Você é um especialista jurídico com foco em simplificação de textos legais. Sua tarefa é analisar acórdãos judiciais e gerar um resumo simplificado, acessível ao público geral, sem perder a precisão jurídica. O formato do documento simplificado deve ser claro, objetivo e seguir uma estrutura pré-definida.
    Preencha o acórdão simplificado utilizando as informações fornecidas em cada bloco. Siga o formato abaixo para garantir que todos os elementos necessários sejam cobertos e organizados conforme a estrutura do acórdão:

    ### FORMATO
    ```
    ### **Resultado do Julgamento (Dispositivo):**

    Resultado final da decisão: Informe de maneira direta e clara o resultado final da decisão, incluindo em 2 parágrafos, com no máximo 3 linhas em cada um, em texto contínuo.
    Resultado do Julgamento: Aqui, deverá ser informado o resultado final da decisão, se a decisão foi unânime ou não, se manteve a decisão original ou a modificou. Exemplo: “O Tribunal Regional Eleitoral do Ceará, por unanimidade, decidiu manter a decisão original, entendendo que...".

    Decisão: de acordo com o resultado da decisão, uma entre: "Recurso Negado", "Recurso Aceito" ou "Recurso Parcialmente Aceito".
    ```
    ### FIM DO FORMATO

    ### CONTEXTO
    ```
    {context}
    ```
    ### FIM DO CONTEXTO

    Resumo simplificado:
    """
)

### Recebendo a resposta

In [36]:
result = stuff(doc, 0, results, 'decisao', decisao_prompt)

### **Resultado do Julgamento (Dispositivo):**

O Tribunal Regional Eleitoral do Ceará decidiu negar provimento ao recurso do candidato Antônio Ilomar Vasconcelos Cruz, que alegou que havia realizado propaganda eleitoral antecipada durante o lançamento de sua pré-candidatura.

A decisão foi tomada por unanimidade e é baseada no voto do relator. O Tribunal considerou que o ato da propaganda eleitoral antecipada foi praticado pelo recorrente, o que não está abrangido pelo rol do artigo 36-A da Lei 9.504/97.

### Decisão: Recurso Negado

O recurso foi negado provimento, de acordo com a decisão do Tribunal Regional Eleitoral do Ceará.


In [37]:
prompts = f"""# Cabeçalho

{results['cabecalho']['prompt']}

# Relatório

{results['relatorio']['prompt']}

# Voto

{results['voto']['prompt']}

# Decisão

{results['decisao']['prompt']}
"""
print(len(prompts))
pprint(prompts)

6926
('# Cabeçalho\n'
 '\n'
 '["input_variables=[\'context\'] input_types={} partial_variables={} '
 "template='\\\\n    Extraia do texto encontrado no contexto somente as "
 'seguintes informações:\\\\n    \\\\n    -   Número do processo (ou número do '
 'recurso eleitoral)\\\\n    -   Origem\\\\n    -   Relator\\\\n    -   '
 'Recorrente\\\\n    -   Advogados(as)\\\\n    -   Recorrido\\\\n    -   '
 'Assunto\\\\n    \\\\n    Formate sua resposta como uma lista, assim:\\\\n    '
 '\\\\n    -   Número do processo: Número do processo\\\\n    -   Origem: '
 'Origem\\\\n    -   Relator: Relator\\\\n    -   Recorrente: '
 'Recorrente\\\\n    -   Advogados: Advogados(as)\\\\n    -   Recorrido: '
 'Recorrido\\\\n    -   Assunto: Assunto, em 2 ou 3 palavras\\\\n    \\\\n    '
 '### Exemplo de entrada:\\\\n    \\\\n    ```\\\\n    PODER '
 'JUDICIÁRIO\\\\n    TRIBUNAL REGIONAL ELEITORAL DO CEARÁ\\\\n    \\\\n    '
 'RECURSO ELEITORAL N. 0600012-49.2024.6.06.0033\\\\n    ORIGEM: '
 'CANINDÉ/CE\

In [ ]:
results

In [39]:
acordao = f"""**Tribunal Regional Eleitoral do Ceará**

{results['cabecalho']['response']}

{results['relatorio']['response']}

{results['voto']['response']}

{results['decisao']['response']}
"""
print(acordao)

**Tribunal Regional Eleitoral do Ceará**

    -   Número do processo: 0600012-49.2024.6.06.0033
    -   Origem: CANINDÉ/CE
    -   Relator: DESEMBARGADOR ELEITORAL DANIEL CARVALHO CARNEIRO
    -   Recorrente: ANTÔNIO ILOMAR VASCONCELOS CRUZ
    -   Advogados(as): FRANCISCO JARDEL RODRIGUES DE SOUSA - OAB CE32787-A, LIDENIRA CAVALCANTE MENDONÇA VIEIRA - OAB CE0016731
    -   Recorrido: MINISTÉRIO PÚBLICO ELEITORAL
    -   Assunto: Propaganda eleitoral antecipada

**Relatório (O Caso)**

O caso envolve uma ação eleitoral contra o candidato Antônio Ilomar Vasconcelos Cruz, que foi acusado de realizar propaganda eleitoral antecipada antes do período oficial de campanha. O recurso foi apresentado pelo Ministério Público Eleitoral e pediu a negação do provimento do recurso.

**Informações iniciais do processo**

O autor do recurso pediu que o Tribunal Regional Eleitoral do Ceará negasse provimento ao recurso, argumentando que não havia sido cometido ato típico de propaganda eleitoral antecip

In [ ]:
simplificar_prompt = PromptTemplate.from_template("""
    Simplifique o texto, mantendo sua estrutura original e seguindo as seguintes diretivas:

    1. Utilize sempre uma linguagem simples e direta, evitando termos técnicos desnecessários. Caso seja imprescindível usar um termo jurídico específico, certifique-se de explicá-lo claramente.
    2. Mantenha o número de parágrafos e linhas conforme indicado para cada seção.
    3. Assegure-se de que as informações jurídicas sejam precisas e que a fundamentação legal esteja diretamente relacionada ao caso em discussão.
    4. Siga as regras de linguagem simples: seja empático e cordial, use frases curtas, escreva na ordem direta e em voz ativa, coloque as informações mais importantes no início e utilize verbos de ação.
    5. Revise o texto para garantir que seja claro e acessível, evitando siglas, jargões e termos técnicos. Confirme se todas as diretrizes e restrições foram cumpridas.
    6. Adote uma escrita humanizada, procurando aproximar o texto da forma como um ser humano se expressaria, para tornar a comunicação mais próxima e compreensível.

    ### CONTEXTO

    ```
    {context}
    ```

    ### RESPOSTA
""")

In [ ]:
simplificar_chain = simplificar_prompt | get_llama(0)

In [ ]:
result = simplificar_chain.invoke({'context': acordao})

In [ ]:
pprint(result.content)

In [25]:
refinar_resposta_prompt = PromptTemplate.from_template(
    """
    Você é um especialista jurídico com foco em revisão e correção de textos legais. Sua tarefa é revisar a resposta gerada anteriormente, comparando-a com o texto original, e corrigir quaisquer inconsistências factuais. Certifique-se de que todas as informações estejam corretas e precisas.

    ### Resposta Anterior
    ```
    {resumo}
    ```
    ### Fim da Resposta Anterior

    ### Texto Original
    ```
    {context}
    ```
    ### Fim do Texto Original

    ### Instruções
    1. Leia a resposta anterior e o texto original cuidadosamente.
    2. Identifique quaisquer inconsistências factuais ou informações incorretas na resposta anterior.
    3. Corrija as inconsistências e forneça uma resposta revisada e precisa.
    4. Mantenha a estrutura e o formato da resposta original, mas assegure-se de que todas as informações estejam corretas.

    ### Resposta Revisada
    """
)
avaliacao = {
    'relatorio': {},
}
avaliar_chain = stuff(doc, 0, avaliacao, 'relatorio', refinar_resposta_prompt, {'resumo': results['relatorio']['response']})


Prompt: Human: 
    Você é um especialista jurídico com foco em revisão e correção de textos legais. Sua tarefa é revisar a resposta gerada anteriormente, comparando-a com o texto original, e corrigir quaisquer inconsistências factuais. Certifique-se de que todas as informações estejam corretas e precisas.

    ### Resposta Anterior
    ```
    **Relatório (O Caso)**: O caso envolve um pré-candidato a prefeito do município de Canindé, que realizou uma campanha eleitoral antecipada. Ele foi condenado a pagar multa de R$ 15.000 por violar o artigo 36-A da Lei 9.504/97.

**Informações iniciais do processo analisado pelo juiz**: A Promotoria Eleitoral acusou o pré-candidato de realizar propaganda eleitoral antecipada, com pedido explícito de votos e ampla divulgação em redes sociais. O pré-candidato argumentou que estava apenas exercendo seu direito à liberdade de expressão e divulgação da pré-candidatura.

**Decisão do juiz no processo inicial**: O juízo a quo condenou o pré-candidato ao

KeyboardInterrupt: 